# Библиотеки

In [1]:
import neptune
import pandas as pd

from cv import get_indices
from load_data import load
from model_selection import training
import random
from NN import simple_torchpl
from pl_framework import nn_training
from pytorch_forecasting.metrics import MAPE
import torch

In [2]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [3]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="3334104b-e4da-40ca-a353-d78790164c0b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [4]:
data_path = 'data/'
random_state = 54321

# Загрузка данных

In [5]:
train_features, train_targets, _ = load(data_path)

# Валидация

In [6]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [7]:
#train_targets = train_targets.drop('timestamp', axis = 'columns')
#train_features = train_features.drop('timestamp', axis = 'columns')

# Подбор модели

In [ ]:
my_training = training(name = 'NN', nn_model = simple_torchpl, training_nn = nn_training,
                       description = 'Ffill. 1 train set for all folds. 1 model for 1 target. simple NN, 1 linear layers\
                       Loss = MAE.Separate shift for each feature. seq_len = 1.180-220 range for shift. No shift in rates.\
                       No normalisation. Only first 2 fold and test set(last fold).Dropped data before 2020-02-15 00:00:00',
                       upload_source_files = ['cv.py', 
                                              'load_data.py', 
                                              'model_selection.py',
                                              'NN.py',
                                              'pl_framework.py',
                                             'preprocesssing.py'])
my_training.set_up_studying(random_state = random_state)

model = 'torch'

def params_func(trial, X):
    return(
        
{        
            'optimizer': 'AdamW',
            'lr': trial.suggest_loguniform('lr', 0.0001, 0.2),
            'weight_decay': trial.suggest_uniform('weight_decay', 0.0001, 1),
            'loss': torch.nn.L1Loss,
            'batch_size': trial.suggest_int('batch_size', 10, 540),
            'p_1': trial.suggest_uniform('p_1', 0, 1),
            'p_2': trial.suggest_uniform('p_2', 0, 1),
            'n_in': 0,
            'n_out': 1,
            'activation1': trial.suggest_categorical('activation1', ['Tanh','Hardtanh','Hardshrink', 'ELU' , 
                                                                  'SELU', 'ReLU', 'Tanhshrink', 'CELU']),
            'n_h_1': trial.suggest_int('n_h_1', 150, 1024),
            'n_back_A_CH4': trial.suggest_int('n_back_A_CH4', 180, 220),
            'n_back_A_C2H6': trial.suggest_int('n_back_A_C2H6', 180, 220),
            'n_back_A_C3H8': trial.suggest_int('n_back_A_C3H8', 180, 220),
            'n_back_A_iC4H10': trial.suggest_int('n_back_A_iC4H10', 180, 220),
            'n_back_A_iC5H12': trial.suggest_int('n_back_A_iC5H12', 180, 220),
            'n_back_A_nC4H10': trial.suggest_int('n_back_A_nC4H10', 180, 220),
            'n_back_A_nC5H12': trial.suggest_int('n_back_A_nC5H12', 180, 220),
            'n_back_A_C6H14': trial.suggest_int('n_back_A_C6H14', 180, 220)
 
}
    )

n_trials = 100_000
my_training.train(X = train_features, 
                  y = train_targets, 
                  cv = cv, 
                  model=model, 
                  params_func = params_func, 
                  n_trials = n_trials)

https://ui.neptune.ai/iliaavilov/SIBUR/e/SIB-313


[I 2020-12-01 21:18:47,054] A new study created in memory with name: no-name-53250dae-37f4-43b5-ad89-14897c80ee40


current params: {'optimizer': 'AdamW', 'lr': 0.1021770986489331, 'weight_decay': 0.6238624537060861, 'loss': <class 'torch.nn.modules.loss.L1Loss'>, 'batch_size': 160, 'p_1': 0.4297847966684789, 'p_2': 0.5430129619658469, 'n_in': 0, 'n_out': 1, 'activation1': 'CELU', 'n_h_1': 672, 'n_back_A_CH4': 214, 'n_back_A_C2H6': 191, 'n_back_A_C3H8': 218, 'n_back_A_iC4H10': 192, 'n_back_A_iC5H12': 210, 'n_back_A_nC4H10': 187, 'n_back_A_nC5H12': 191, 'n_back_A_C6H14': 193}
X shape before dropped nans (3647, 11)
cv before trans: 719
Initial shape of X after dropping nans: (3429, 11)
Initial shape of y: (3647, 5)
cv after trans: 501
shape of X after trans: (3429, 10)
shape of y after trans: (3429, 5)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type    | Params
-----------------------------------------
0 | activ1       | CELU    | 0     
1 | dropout1     | Dropout | 0     
2 | dropout2     | Dropout | 0     
3 | linear1      | Linear  | 7.4 K 
4 | linear_final | Linear  | 673   
5 | loss         | L1Loss  | 0     
6 | loss_sec     | MAPE    | 0     
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type    | Params
-----------------------------------------
0 | activ1       | CELU    | 0     
1 | dropout1     | Dropout | 0     
2 | dropout2     | Dropout | 0     
3 | linear1      | Linear  | 7.4 K 
4 | linear_final | Linear  | 673   
5 | loss         | L1Loss  | 0     
6 | loss_sec     | MAPE    | 0     
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICE

current params: {'optimizer': 'AdamW', 'lr': 0.07409214003577125, 'weight_decay': 0.012276509940363431, 'loss': <class 'torch.nn.modules.loss.L1Loss'>, 'batch_size': 384, 'p_1': 0.7367198950145076, 'p_2': 0.8220563827251418, 'n_in': 0, 'n_out': 1, 'activation1': 'Tanhshrink', 'n_h_1': 897, 'n_back_A_CH4': 208, 'n_back_A_C2H6': 180, 'n_back_A_C3H8': 203, 'n_back_A_iC4H10': 198, 'n_back_A_iC5H12': 185, 'n_back_A_nC4H10': 202, 'n_back_A_nC5H12': 181, 'n_back_A_C6H14': 209}
X shape before dropped nans (3647, 11)
cv before trans: 719
Initial shape of X after dropping nans: (3438, 11)
Initial shape of y: (3647, 5)
cv after trans: 510
shape of X after trans: (3438, 10)
shape of y after trans: (3438, 5)


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | activ1       | Hardshrink | 0     
1 | dropout1     | Dropout    | 0     
2 | dropout2     | Dropout    | 0     
3 | linear1      | Linear     | 2.1 K 
4 | linear_final | Linear     | 196   
5 | loss         | L1Loss     | 0     
6 | loss_sec     | MAPE       | 0     
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | activ1       | Hardshrink | 0     
1 | dropout1     | Dropout    | 0     
2 | dropout2     | Dropout    | 0     
3 | linear1      | Linear     | 2.1 K 
4 | linear_final | Linear     | 196   
5 | loss         | L1Loss     | 0     
6 | loss_sec     | MAPE       | 0     
GPU available: True, used: True
TPU available: False, 

current params: {'optimizer': 'AdamW', 'lr': 0.0008589167620028695, 'weight_decay': 0.9994933905360369, 'loss': <class 'torch.nn.modules.loss.L1Loss'>, 'batch_size': 376, 'p_1': 0.6556090407197952, 'p_2': 0.8742602783327769, 'n_in': 0, 'n_out': 1, 'activation1': 'Hardtanh', 'n_h_1': 150, 'n_back_A_CH4': 211, 'n_back_A_C2H6': 206, 'n_back_A_C3H8': 200, 'n_back_A_iC4H10': 209, 'n_back_A_iC5H12': 189, 'n_back_A_nC4H10': 199, 'n_back_A_nC5H12': 190, 'n_back_A_C6H14': 209}
X shape before dropped nans (3647, 11)
cv before trans: 719
Initial shape of X after dropping nans: (3436, 11)
Initial shape of y: (3647, 5)
cv after trans: 508
shape of X after trans: (3436, 10)
shape of y after trans: (3436, 5)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type     | Params
------------------------------------------
0 | activ1       | Hardtanh | 0     
1 | dropout1     | Dropout  | 0     
2 | dropout2     | Dropout  | 0     
3 | linear1      | Linear   | 1.7 K 
4 | linear_final | Linear   | 151   
5 | loss         | L1Loss   | 0     
6 | loss_sec     | MAPE     | 0     
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type     | Params
------------------------------------------
0 | activ1       | Hardtanh | 0     
1 | dropout1     | Dropout  | 0     
2 | dropout2     | Dropout  | 0     
3 | linear1      | Linear   | 1.7 K 
4 | linear_final | Linear   | 151   
5 | loss         | L1Loss   | 0     
6 | loss_sec     | MAPE     | 0     
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - C

current params: {'optimizer': 'AdamW', 'lr': 0.00018110310168866685, 'weight_decay': 0.9634594962468537, 'loss': <class 'torch.nn.modules.loss.L1Loss'>, 'batch_size': 346, 'p_1': 0.5646642647789857, 'p_2': 0.6937582377514128, 'n_in': 0, 'n_out': 1, 'activation1': 'Hardtanh', 'n_h_1': 273, 'n_back_A_CH4': 220, 'n_back_A_C2H6': 203, 'n_back_A_C3H8': 190, 'n_back_A_iC4H10': 183, 'n_back_A_iC5H12': 192, 'n_back_A_nC4H10': 212, 'n_back_A_nC5H12': 185, 'n_back_A_C6H14': 199}
X shape before dropped nans (3647, 11)
cv before trans: 719
Initial shape of X after dropping nans: (3427, 11)
Initial shape of y: (3647, 5)
cv after trans: 499
shape of X after trans: (3427, 10)
shape of y after trans: (3427, 5)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type     | Params
------------------------------------------
0 | activ1       | Hardtanh | 0     
1 | dropout1     | Dropout  | 0     
2 | dropout2     | Dropout  | 0     
3 | linear1      | Linear   | 3.0 K 
4 | linear_final | Linear   | 274   
5 | loss         | L1Loss   | 0     
6 | loss_sec     | MAPE     | 0     


In [ ]:
#neptune.log_metric('leaderboard_mape', 3.2443)
#neptune.project.get_experiments('SIB-257')[0].stop()
neptune.stop()